# Seminar Project: Preprocessing Articles
---------------
``` 

Contributors:
- Elias Heppner

***

This notebook gets the articles from https://github.com/kssrr/german-media-scrape and analyzes the most frequent topics. It was the basis for our choice of prompting topics.

### Loading Packages


In [1]:
import numpy as np 
import pandas as pd 
import time
import pickle
import time
import os
import datetime
from tqdm.auto import tqdm
import matplotlib.pyplot as plt

# probably wise to switch to tiktoken as tokenizer as this is the one recommended by OpenAI 
import nltk
nltk.download('punkt')  # Nur notwendig beim ersten Mal
from nltk.tokenize import word_tokenize # import tokenizer to estimate token count of a speech


c:\Users\elias\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\elias\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### Load Article Data


In [2]:
#Define 
os.chdir("../data/articles")
os.getcwd()

'c:\\Users\\elias\\Documents\\Master\\3. Semester\\LLM\\Politricks\\data\\articles'

In [3]:
import tarfile

file_path = "data.tar.gz"

# Extract the contents of the tar.gz file
with tarfile.open(file_path, "r:gz") as tar:
    # Extract all contents to a temporary directory
    tar.extractall(path="temp")

# List the contents of the extracted directory
extracted_files = os.listdir("temp")
extracted_files

['data']

In [4]:
spgl_df = pd.read_csv("temp/data/spiegel.csv")
sz_df = pd.read_csv("temp/data/sz.csv")
taz_df = pd.read_csv("temp/data/taz.csv")
welt_df = pd.read_csv("temp/data/welt.csv")
zeit_df = pd.read_csv("temp/data/zeit.csv")

#os.rmdir("temp")

C:\Users\elias\AppData\Local\Temp\ipykernel_6604\749409521.py:5: DtypeWarning: Columns (7,9) have mixed types. Specify dtype option on import or set low_memory=False.
  zeit_df = pd.read_csv("temp/data/zeit.csv")


In [5]:
spgl_df

,Unnamed: 0,X.,url,date,author,description,keywords,paywall,error,body
0,1,1,https://www.spiegel.de/ausland/amtsenthebungsv...,2020-01-31T23:55:00+01:00,DER SPIEGEL,John Bolton wird US-Präsident Donald Trump nic...,"Ausland, Impeachment von Donald Trump, Donald ...",0.0,NaN,Donald Trump Brendan Smialowski / AFP Im Amtse...
1,2,2,https://www.spiegel.de/sport/fussball/daniel-f...,2020-01-31T23:50:14+01:00,DER SPIEGEL,"Daniel Frahn musste beim Chemnitzer FC gehen, ...","Sport, Fußball-News",0.0,NaN,Daniel Frahn im Trikot des Chemnitzer FC Danie...
2,3,3,https://www.spiegel.de/sport/eishockey/eishock...,2020-01-31T23:09:06+01:00,DER SPIEGEL,Die Kölner Haie gingen als Geheimfavorit in di...,"Sport, Eishockey, Eishockey",0.0,NaN,Die historische Niederlagenserie der Kölner Ha...
3,4,4,https://www.spiegel.de/ausland/syrien-krieg-ba...,2020-01-31T23:08:03+01:00,"Alexander Sarovic, DER SPIEGEL",Mithilfe Russlands führt der syrische Diktator...,"Ausland, Bürgerkrieg in Syrien, Syrien, Flücht...",0.0,NaN,Offensive des Assad-Regimes in Idlib: Hundertt...
4,5,5,https://www.spiegel.de/ausland/usa-donald-trum...,2020-01-31T23:07:30+01:00,DER SPIEGEL,Barack Obama erlaubte den Einsatz von Landmine...,"Ausland, Waffen, USA, Landminen",0.0,NaN,"Die Suche nach Landminen, wie hier in Kandahar..."
...,...,...,...,...,...,...,...,...,...,...
118764,118765,118765,https://www.spiegel.de/ausland/emmanuel-macron...,2022-12-01T04:16:54+01:00,DER SPIEGEL,Heute trifft sich Emmanuel Macron mit Joe Bide...,"Ausland, Emmanuel Macron, Joe Biden, USA, Wash...",0.0,NaN,Emmanuel Macron (l.) und Joe Biden beim Verlas...
118765,118766,118766,https://www.spiegel.de/wirtschaft/unternehmen/...,2022-12-01T02:59:46+01:00,DER SPIEGEL,Erstmals seit dem spektakulären Zusammenbruch ...,"Wirtschaft, Unternehmen, Kryptowährungen, USA,...",0.0,NaN,"Samuel Bankman-Fried, in der Kryptobranche oft..."
118766,118767,118767,https://www.spiegel.de/kultur/kino/ang-lee-dre...,2022-12-01T02:13:46+01:00,DER SPIEGEL,Bruce Lee machte das Martial-Arts-Kino aus Hon...,"Kultur, Kino, Ang Lee, Hollywood, Film, Kampfs...",0.0,NaN,Regisseur Ang Lee (2016 in Taiwan): Es bleibt ...
118767,118768,118768,https://www.spiegel.de/ausland/donald-trump-us...,2022-12-01T01:26:35+01:00,DER SPIEGEL,Donald Trump hatte bis zum obersten US-Gericht...,"Ausland, Donald Trump, Republikaner (USA), Dem...",0.0,NaN,Donald Trump (am 15. November) Das US-Finanzmi...


In [6]:
sz_df

,Unnamed: 0,X.,url,date,title,author,description,keywords,paywall,error,body
0,1,1,https://www.sueddeutsche.de/muenchen/ebersberg...,2020-01-31 21:49:34,"Exportschlager aus Kirchseeon - Von einem, der...",Süddeutsche Zeitung,"Mario Lochert, Bassist der Metal-Band ""Serious...","Kultur im Landkreis Ebersberg,Kirchseeon,Landk...",False,NaN,"Scheinwerferlicht, schwarze Klamotten, voll tä..."
1,2,2,https://www.sueddeutsche.de/stil/restaurant-an...,2020-01-12 17:10:00,Sterne-Restaurant: Oberkellner über ihre Gäste,Jan Schwenkenbecher,"Zwei Oberkellner berichten anonym, wie Mensche...","Gastronomie,Essen und Trinken,Gourmet,Guide Mi...",True,NaN,NaN
2,3,3,https://www.sueddeutsche.de/medien/wechsel-von...,2020-01-14 18:46:05,Wechsel von Rheinischer Post - Neue Vize-Chefi...,Süddeutsche Zeitung,Die stellvertretende Chefredakteurin der Rhein...,"Medien,Rheinische Post,Medien,Süddeutsche Zeitung",False,NaN,Die stellvertretende Chefredakteurin der Rhein...
3,4,4,https://www.sueddeutsche.de/medien/schweizer-o...,2020-01-13 18:27:18,Schweizer Öffentlich-Rechtliche - Streitfall,Isabel Pfaff,Bekommt die Schweizer Radio- und Fernsehgesell...,"Medien,NZZ,Medien,Süddeutsche Zeitung",False,NaN,Schweizer Politiker debattieren über die finan...
4,5,5,https://www.sueddeutsche.de/politik/jugendamt-...,2020-01-01 13:05:38,Eine Inobhutnahme darf kein zufälliges Schicks...,Süddeutsche Zeitung,"Immer wieder werden Alleinerziehende, die beim...","Familienpolitik,Familie,Kindeswohl,Meinung,Pol...",False,NaN,Für das neue Jahr plant die große Koalition ei...
...,...,...,...,...,...,...,...,...,...,...,...
257675,257676,257676,https://www.sueddeutsche.de/muenchen/dachau/ha...,2022-12-01 09:09:06,Verschuldet und depressiv: SZ-Adventskalender ...,Süddeutsche Zeitung,Maria R. hat Schulden und leidet an einer Depr...,"Landkreis Dachau,Haimhausen,SZ-Adventskalender...",False,NaN,Im Leben der 65-jährigen Maria R. (Name von de...
257676,257677,257677,https://www.sueddeutsche.de/politik/news-ukrai...,2022-12-01 07:00:31,News am Morgen - Nachrichten vom 01. Dezember ...,Leopold Zaak,Bundeswirtschaftsministerium plant neue China-...,"SZ Nachrichten des Tages,Politik,Süddeutsche Z...",False,NaN,Habeck-Beamte rechnen mit Annexion Taiwans dur...
257677,257678,257678,https://www.sueddeutsche.de/politik/usa-tuerke...,2022-12-01 03:28:42,USA warnen Erdoğan vor neuer Militärkampagne i...,Süddeutsche Zeitung,Die Türkei hat kurdische Gebiete in Syrien bom...,"Türkei,Anschlag in Istanbul,Kurden,PKK,Syrien,...",False,NaN,Die USA haben die Türkei vor einer neuen Milit...
257678,257679,257679,https://www.sueddeutsche.de/kultur/metallica-l...,2022-12-01 03:32:47,"Neue Single, neue Tour: Metallica sind zurück",Andrian Kreye,"Neue Single, neue Tour: ""Metallica"" sind zurüc...","Pop,Heavy Metal,Lux Aeterna,Musik,Requiem,Rock...",True,NaN,NaN


In [7]:
taz_df

,Unnamed: 0,X.,url,date,title,author,description,keywords,body,error
0,1,1,https://taz.de/Neues-Buch-aus-Nachlass-von-Imr...,2022-07-30T12:57:00+01:00,Neues Buch aus Nachlass von Imre Kertész: Das ...,Julia Hubernagel,Sechs Jahre nach seinem Tod ist ein Arbeitstag...,"Nobelpreis für Literatur, Literatur, Buch, Tag...","Imre Kertész, geboren 1929 in Budapest, hat di...",NaN
1,2,2,https://taz.de/Ampel-einigt-sich-auf-Wahlrecht...,2022-07-06T15:01:00+01:00,Ampel einigt sich auf Wahlrechtsreform: Der Bu...,Sabine am Orde,Die Ampel hat sich auf Eckpunkte einer Wahlrec...,"Wahlrechtsreform, Bundestag, Ampel-Koalition, ...",BERLIN taz | Die Ampel-Fraktionen wollen bis z...,NaN
2,3,3,https://taz.de/Messerangriff-in-ICE-bei-Regens...,2022-07-25T16:35:00+01:00,Messerangriff in ICE bei Regensburg: Anklage w...,Konrad Litschko,Ein Mann attackierte in einem ICE vier Männer ...,"Islamist, Islamismus, „Islamischer Staat“ (IS)...",BERLIN taz | Es war eine brutale Attacke. Am 6...,NaN
3,4,4,https://taz.de/Streit-um-Wahlrechtsreform/!586...,2022-07-06T17:29:00+01:00,Streit um Wahlrechtsreform: Abstriche an falsc...,Gereon Asmuth,Die Ampel will den Bundestag abspecken und Gew...,"Wahlrechtsreform, Bundesregierung, Ampel-Koali...",Die Ampelkoalition will allen Ernstes nicht me...,NaN
4,5,5,https://taz.de/Waldbraende-in-den-USA-und-Euro...,2022-07-24T14:41:00+01:00,Waldbrände in den USA und Europa: 6.000 Mensch...,taz. die tageszeitung,In Kalifornien breiten sich Feuer aus. Der Wal...,"Waldbrand, Klimawandel, Klima, Brand, Dürre, H...",LOS ANGELES/MARIPOSA/ATHEN/PARIS/SANTA CRUZ DE...,NaN
...,...,...,...,...,...,...,...,...,...,...
70797,70798,70798,https://taz.de/Trumps-Nahost-Friedensplan/!565...,2020-01-31T18:24:00+01:00,Trumps Nahost-Friedensplan: Nur heiße Luft,Susanne Knaul,Was auch immer US-Präsident Trump mit seinem s...,"Donald Trump, Palästina, Nahost-Konflikt, Naho...","Jason Greenblatt, US-Sondergesandter und Partn...",NaN
70798,70799,70799,https://taz.de/117-neue-Emojis/!5656354/,2020-01-31T18:25:00+01:00,117 neue Emojis: Aber was willst du denn? 🤔,Malte Göbel,Erleichtern Emojis die Kommunikation? Nicht wi...,"Emojis, Kommunikation, Smartphone, Netzkultur,...",Zugegeben: Unter den 117 neuen Emojis gibt es ...,NaN
70799,70800,70800,https://taz.de/Klimakrise-und-Hysterie/!5656491/,2020-01-31T18:25:00+01:00,Klimakrise und Hysterie: Im Shitstorm für den ...,Laura Sophia Jung,Der Klimawandel ist nicht aufzuhalten. Die Fix...,"Klima, Jonathan Franzen, Klimawandel, Buch, Ku...","Anders als viele Autor*innen, die sich in den ...",NaN
70800,70801,70801,https://taz.de/Mietendeckel-in-Berlin/!5656404/,2020-01-31T18:46:00+01:00,Mietendeckel in Berlin: Nur die halbe Miete,Ingo Arzt,Der Berliner Mietendeckel ist für die Mieter*i...,"Mietendeckel, Wärmedämmung, Energetische Sanie...",An anderen Ecken in Berlin finden sich extreme...,NaN


In [8]:
welt_df

,Unnamed: 0,X.,url,date,time,author,description,keywords,paywall,error,body
0,1,1,https://www.welt.de/reise/Fern/article20441783...,2020-01-01,07:42:11,Kira Hanser,"Es gibt Bahnstrecken, auf denen der Zug gar ni...","Hanser-Kira, Bahnreisen, Schweiz, Schottland, ...",0,NaN,Acht Stunden Ausblick total: Bei der Panoramaf...
1,2,2,https://www.welt.de/kmpkt/article204460512/Foo...,2020-01-01,07:43:26,NaN,"Wer auf sich hält, kocht selbst. Und wer auf d...","Instagram, Nagels-Philipp, Ernährung, Internet...",0,NaN,"Seien wir ehrlich, Hochglanz ist doch langweil..."
2,3,3,https://www.welt.de/iconist/essen-und-trinken/...,2020-01-01,07:42:40,Heiko Zwirner,"Mit deftigen Gerichten, französischen Weinen u...","Einzelne Restaurants, Rotwein, Zwirner-Heiko, ...",0,NaN,"Keine Musik, keine Kunst an den Wänden, keine ..."
3,4,4,https://www.welt.de/sport/plus170079992/Die-un...,2020-01-01,23:45:00,Lutz Wöckener,Der Schotte Peter Wright stammt aus ärmlichen ...,"Woeckener-Lutz, Taylor, Phil (Dartspieler), Da...",1,NaN,NaN
4,5,5,https://www.welt.de/politik/deutschland/plus20...,2020-01-01,11:51:29,Kaja Klapsa,"In Deutschland kaum möglich, in der Schweiz we...","Klapsa-Kaja, Sterbehilfe, Schweiz, Tod, Krankh...",1,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
593833,593834,593834,https://www.welt.de/wirtschaft/webwelt/article...,2022-12-31,00:44:48,Thomas Heuzeroth,"Von Computerviren, über Identitätsdiebstahl bi...","Heuzeroth-Thomas, Bitkom, Apple Podcast, Amazo...",0,NaN,Der überwiegende Teil der Internetnutzer in De...
593834,593835,593835,https://www.welt.de/podcasts/alles-auf-aktien/...,2022-12-31,04:11:40,Holger Zschäpitz,In dieser Samstags-Episode von “Alles auf Akti...,"Eckert-Daniel, Sommerfeldt-Nando, Inflation, F...",0,NaN,In dieser Samstags-Episode von “Alles auf Akti...
593835,593836,593836,https://www.welt.de/kultur/medien/article24296...,2022-12-31,03:57:51,NaN,US-Fernsehjournalistin Barbara Walters ist tot...,"Newsteam, Walters, Barbara, Barbara Walter, Wl...",0,NaN,Die berühmte US-Fernsehjournalistin Barbara Wa...
593836,593837,593837,https://www.welt.de/newsticker/dpa_nt/infoline...,2022-12-31,02:44:17,NaN,"Es ist noch nicht Mitternacht, trotzdem wird s...","Frankfurt am Main, Silvesterrakete, Brand, Feu...",0,NaN,Eine fehlgeleitete Silvesterrakete hat ein Feu...


In [9]:
zeit_df

,Unnamed: 0,url,date,title,description,keywords,author,paywall,body,error
0,1,https://www.zeit.de/news/2020-01/01/kalenderbl...,2020-01-01T23:55:00+01:00,Was geschah am ...: Kalenderblatt 2020: 2. Jan...,Hier finden Sie Informationen zu dem Thema „Wa...,"News, Was geschah am ..., Dänemark, Europa, Sü...",NaN,False,Berlin (dpa) - Das aktuelle Kalenderblatt für ...,NaN
1,2,https://www.zeit.de/news/2020-01/01/deutsche-h...,2020-01-01T23:47:28+01:00,Handball: Deutsche Handballer starten in direk...,Hier finden Sie Informationen zu dem Thema „Ha...,"News, Handball, Frankfurt am Main, Deutschland...",NaN,False,Frankfurt/Main (dpa) - Deutschlands Handballer...,NaN
2,3,https://www.zeit.de/news/2020-01/01/guardiola-...,2020-01-01T23:00:08+01:00,Premier League: Guardiola gewinnt Duell der Ex...,Hier finden Sie Informationen zu dem Thema „Pr...,"News, Premier League, Carlo Ancelotti, Gabriel...",NaN,False,Manchester (dpa) - Pep Guardiola hat mit Manch...,NaN
3,4,https://www.zeit.de/news/2020-01/01/peter-wrig...,2020-01-01T22:55:59+01:00,Darts: Peter Wright erobert Darts-Krone - 7:3-...,Hier finden Sie Informationen zu dem Thema „Da...,"News, Darts, Peter Wright, Michael van Gerwen,...",NaN,False,London (dpa) - Paradiesvogel Peter Wright hat ...,NaN
4,5,https://www.zeit.de/news/2020-01/01/zeichen-in...,2020-01-01T22:52:00+01:00,Regierung: Zeichen in Österreich stehen auf Sc...,Hier finden Sie Informationen zu dem Thema „Re...,"News, Regierung, Österreich, Wien, ÖVP, FPÖ",NaN,False,Wien (dpa) - Die konservative ÖVP und die Grün...,NaN
...,...,...,...,...,...,...,...,...,...,...
443001,443002,https://www.zeit.de/2023/01/kinder-jugendliche...,2022-12-31T10:05:11+01:00,Kinder und Jugendliche: Klein und allein | ZEI...,"2022 endet mit Alarmrufen aus Kliniken, Kitas ...","Gesellschaft, Kinder und Jugendliche, Gesundhe...",Johanna Schoener,False,Kinder fallen in die Kategorie low hanging fru...,NaN
443002,443003,https://www.zeit.de/2023/01/erschoepfung-psych...,2022-12-31T09:27:29+01:00,Psychologie: Irgendwie alle | ZEIT ONLINE,So erschöpft wie gerade waren viele noch nie. ...,"Wissen, Psychologie, Psychologie, Fachkräftema...",Anna-Lena Scholz,True,NaN,NaN
443003,443004,https://www.zeit.de/2023/01/ukraine-krieg-ende...,2022-12-31T17:22:30+01:00,"Krieg gegen die Ukraine: ""Wir müssen den Krieg...","""Den Krieg vom Ende her denken"" ist eine popul...","Politik, Krieg gegen die Ukraine, Krieg in der...",Elvira Rosert,True,NaN,NaN
443004,443005,https://www.zeit.de/2023/01/disney-bilanz-bob-...,2022-12-31T19:50:25+01:00,Disney: Prinz Kaschmir ist zurück | ZEIT ONLINE,Disneys Bilanz ist verheerend. Nun soll der fr...,"Wirtschaft, Disney, George Lucas, Hollywood, U...",Heike Buchter,True,NaN,NaN


## Cleaning and Omitting

In [10]:
def clean_articles(df):
    if "paywall" in df.columns:
        df = df[df['paywall'] != True]
        df = df[df['paywall'] != 1]
    df = df.dropna(subset=['body'])
    if "title" not in df.columns:
        df["title"] = None
    df = df.loc[:, ["url", "date", "title", "author", "description", "keywords", "body"]]
    return df

In [11]:
taz_df = clean_articles(taz_df)
sz_df = clean_articles(sz_df)
zeit_df = clean_articles(zeit_df)
spgl_df = clean_articles(spgl_df)
welt_df = clean_articles(welt_df)

## Get newspaper categories

In [42]:
arts_df = pd.concat([taz_df, sz_df, zeit_df, spgl_df, welt_df], axis = 0)

#arts_df.to_csv("articles.csv", index=False)

In [36]:
art_cats = arts_df['keywords'].dropna().str.split(', ').tolist()
art_cats = [item for sublist in art_cats for item in sublist]
from collections import Counter
cat_counts = Counter(art_cats)
art_cats = sorted(set(art_cats))



In [41]:
print(len(art_cats))

import json
cat_counts
sorted_categories = sorted(cat_counts.items(), key=lambda x: x[1], reverse=True)
sorted_categories ####

with open("category_counts.json", 'w') as json_file:
    json.dump(sorted_categories, json_file)

#corona, covid, medizin?, impfung, inzidenz, neuinfektion
#->corona, covid, impf, inzidenz, infektion

#ukraine, russland, konflikt

#migration, flüchtlinge, flucht, geflüchtete
#-> flücht, migrat, flucht

#umwelt, klimawandel, klimaschutz, natur




532480
